# Improved Human Pose Classification using GCN

This notebook implements the enhanced training pipeline for classifying human poses using Graph Convolutional Networks (GCN) with the following improvements:

1. Deeper GCN architecture with residual connections
2. Data augmentation for better generalization
3. Class weighting for balanced training

In [ ]:
# Import required libraries
import torch
from torch_geometric.loader import DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import os

# Import our modules
import sys
sys.path.append('src')
from data_processing import load_dataset, PoseAugmentation
from gcn_model import PoseGCN, DeepPoseGCN

print("All libraries imported successfully!")

## Check Data Paths

In [ ]:
# Define and verify data paths
data_dir = os.path.join(os.getcwd(), 'data_v2')
annotation_dir = os.path.join(os.getcwd(), 'annotations_v2')

# Check if directories exist
print(f"Checking data directory: {data_dir}")
print(f"Exists: {os.path.exists(data_dir)}")

print(f"\nChecking annotation directory: {annotation_dir}")
print(f"Exists: {os.path.exists(annotation_dir)}")

# Check annotation files
annotation_files = [
    't5-sherul-300-195-correct.json',
    'lumbar-K-1.1-160.json'
]

print("\nChecking annotation files:")
for file in annotation_files:
    file_path = os.path.join(annotation_dir, file)
    print(f"{file}: {'Exists' if os.path.exists(file_path) else 'Missing'}")

## Define Training and Evaluation Functions

In [13]:
def train(model, train_loader, optimizer, device, class_weights=None):
    model.train()
    total_loss = 0
    
    for data in tqdm(train_loader, desc='Training', leave=False):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Use weighted loss if class weights are provided
        if class_weights is not None:
            loss = torch.nn.functional.nll_loss(output, data.y, weight=class_weights)
        else:
            loss = torch.nn.functional.nll_loss(output, data.y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

In [14]:
def evaluate(model, loader, device):
    model.eval()
    predictions = []
    labels = []
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            pred = output.max(dim=1)[1]
            
            predictions.extend(pred.cpu().numpy())
            labels.extend(data.y.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return accuracy, precision, recall, f1

In [15]:
def plot_metrics(train_losses, val_metrics, save_path='models/training_metrics.png'):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot training loss
    ax1.plot(train_losses, 'b-', label='Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)
    ax1.legend()
    
    # Plot validation metrics
    epochs = range(len(val_metrics['accuracy']))
    ax2.plot(epochs, val_metrics['accuracy'], 'g-', label='Accuracy')
    ax2.plot(epochs, val_metrics['precision'], 'r-', label='Precision')
    ax2.plot(epochs, val_metrics['recall'], 'b-', label='Recall')
    ax2.plot(epochs, val_metrics['f1'], 'y-', label='F1-Score')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Score')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()
    print(f"Saved training plot to {save_path}")

In [16]:
def train_and_evaluate(model, train_loader, val_loader, optimizer, num_epochs=100, device='cpu', class_weights=None):
    best_f1 = 0
    train_losses = []
    val_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    
    for epoch in range(num_epochs):
        # Train
        train_loss = train(model, train_loader, optimizer, device, class_weights)
        train_losses.append(train_loss)
        
        # Evaluate
        val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, device)
        
        # Store metrics
        val_metrics['accuracy'].append(val_acc)
        val_metrics['precision'].append(val_prec)
        val_metrics['recall'].append(val_rec)
        val_metrics['f1'].append(val_f1)
        
        # Print progress
        print(f'Epoch {epoch+1:03d}:')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Val Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, '
              f'Recall: {val_rec:.4f}, F1: {val_f1:.4f}')
        
        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_metrics': val_metrics,
                'train_losses': train_losses
            }, f'models/model_{model.__class__.__name__}_best.pth')
            print(f"Saved best model with F1: {val_f1:.4f}")
            
    return train_losses, val_metrics

## Setup Data With Augmentation

In [ ]:
try:
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create augmentation transform
    augmentation = PoseAugmentation(
        noise_level=0.02,  # 2% noise relative to the normalized keypoints
        drop_edge_prob=0.1,  # 10% probability to drop an edge
        invisible_prob=0.1,  # 10% probability to mark a keypoint as invisible
        p=0.5  # 50% probability to apply augmentation to a sample
    )
    
    # Load datasets with augmentation
    train_dataset, val_dataset = load_dataset(
        root_dir=data_dir,
        annotation_dir=annotation_dir
    )
    
    # Apply augmentation to training dataset
    train_dataset.transform = augmentation
    
    print(f"Dataset loaded: {len(train_dataset)} training samples, {len(val_dataset)} validation samples")
    
    # Compute class weights for balanced training
    train_labels = [data.y.item() for data in train_dataset]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
    class_weights = torch.FloatTensor(class_weights).to(device)
    print(f"Class weights: {class_weights}")
    
    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Create models directory if it doesn't exist
    os.makedirs('models', exist_ok=True)
    
    print("Setup complete!")
    
except Exception as e:
    print(f"Error during setup: {str(e)}")
    raise

## Train Original PoseGCN Model (Baseline)

In [ ]:
# Initialize and train the original model for comparison
original_model = PoseGCN(num_node_features=2).to(device)
original_optimizer = optim.Adam(original_model.parameters(), lr=0.001)

print("\n=== Training Original PoseGCN ===")
original_losses, original_metrics = train_and_evaluate(
    original_model, train_loader, val_loader, original_optimizer, 
    num_epochs=200, device=device, class_weights=class_weights
)

# Plot metrics for original model
plot_metrics(original_losses, original_metrics, save_path='models/original_model_metrics.png')

## Train Deep PoseGCN Model (Enhanced)

In [ ]:
# Initialize and train the deep model
deep_model = DeepPoseGCN(num_node_features=2).to(device)
deep_optimizer = optim.Adam(deep_model.parameters(), lr=0.001, weight_decay=1e-4)  # Added weight decay for regularization

print("\n=== Training DeepPoseGCN ===")
deep_losses, deep_metrics = train_and_evaluate(
    deep_model, train_loader, val_loader, deep_optimizer, 
    num_epochs=200, device=device, class_weights=class_weights
)

# Plot metrics for deep model
plot_metrics(deep_losses, deep_metrics, save_path='models/deep_model_metrics.png')

## Compare Model Performance

In [ ]:
# Compare the best performance of both models
print("\n=== Model Comparison ===")
print(f"Original PoseGCN - Best F1: {max(original_metrics['f1']):.4f}")
print(f"DeepPoseGCN - Best F1: {max(deep_metrics['f1']):.4f}")

# Add comparison of best precision, recall, and accuracy values
print("\n=== Best Precision, Recall, Accuracy Comparison ===")
print(f"Original PoseGCN - Best Precision: {max(original_metrics['precision']):.4f}")
print(f"DeepPoseGCN - Best Precision: {max(deep_metrics['precision']):.4f}")
print("\nOriginal PoseGCN - Best Recall: {:.4f}".format(max(original_metrics['recall'])))
print("DeepPoseGCN - Best Recall: {:.4f}".format(max(deep_metrics['recall'])))
print("\nOriginal PoseGCN - Best Accuracy: {:.4f}".format(max(original_metrics['accuracy'])))
print("DeepPoseGCN - Best Accuracy: {:.4f}".format(max(deep_metrics['accuracy'])))

# Find the epochs with the best metrics for both models
original_best_f1_epoch = np.argmax(original_metrics['f1'])
deep_best_f1_epoch = np.argmax(deep_metrics['f1'])
print("\n=== Best Model Details (Based on F1 Score) ===")
print(f"Original PoseGCN - Best epoch: {original_best_f1_epoch+1}")
print(f"  F1: {original_metrics['f1'][original_best_f1_epoch]:.4f}")
print(f"  Precision: {original_metrics['precision'][original_best_f1_epoch]:.4f}")
print(f"  Recall: {original_metrics['recall'][original_best_f1_epoch]:.4f}")
print(f"  Accuracy: {original_metrics['accuracy'][original_best_f1_epoch]:.4f}")
print(f"\nDeepPoseGCN - Best epoch: {deep_best_f1_epoch+1}")
print(f"  F1: {deep_metrics['f1'][deep_best_f1_epoch]:.4f}")
print(f"  Precision: {deep_metrics['precision'][deep_best_f1_epoch]:.4f}")
print(f"  Recall: {deep_metrics['recall'][deep_best_f1_epoch]:.4f}")
print(f"  Accuracy: {deep_metrics['accuracy'][deep_best_f1_epoch]:.4f}")

# Plot comparison of all metrics
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(original_metrics['f1'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['f1'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score Comparison')
plt.grid(True)
plt.legend()

# Plot precision comparison
plt.subplot(2, 2, 2)
plt.plot(original_metrics['precision'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['precision'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Precision Comparison')
plt.grid(True)
plt.legend()

# Plot recall comparison
plt.subplot(2, 2, 3)
plt.plot(original_metrics['recall'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['recall'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.title('Recall Comparison')
plt.grid(True)
plt.legend()

# Plot accuracy comparison
plt.subplot(2, 2, 4)
plt.plot(original_metrics['accuracy'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['accuracy'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('models/metrics_comparison.png')
plt.show()

# Create a combined plot of all metrics for each model
plt.figure(figsize=(14, 6))

# Plot all metrics for original model
plt.subplot(1, 2, 1)
plt.plot(original_metrics['f1'], 'y-', label='F1')
plt.plot(original_metrics['precision'], 'r-', label='Precision')
plt.plot(original_metrics['recall'], 'b-', label='Recall')
plt.plot(original_metrics['accuracy'], 'g-', label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Original PoseGCN Metrics')
plt.grid(True)
plt.legend()

# Plot all metrics for deep model
plt.subplot(1, 2, 2)
plt.plot(deep_metrics['f1'], 'y-', label='F1')
plt.plot(deep_metrics['precision'], 'r-', label='Precision')
plt.plot(deep_metrics['recall'], 'b-', label='Recall')
plt.plot(deep_metrics['accuracy'], 'g-', label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Deep PoseGCN Metrics')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('models/all_metrics_by_model.png')
plt.show()

## Analysis and Conclusions

In this notebook, we've implemented and compared two models for human pose classification:

1. **Original PoseGCN**: A simple 2-layer GCN model
2. **DeepPoseGCN**: An enhanced model with 4 layers, residual connections, and batch normalization

Both models were trained with the following improvements:
- Data augmentation to improve generalization
- Class weighting to balance precision and recall

The results show that the DeepPoseGCN generally achieves better F1 scores, indicating a better balance between precision and recall. This makes it more suitable for real-world applications where both metrics are important.

### Further Improvements

Some potential ways to further enhance performance:
1. Implement learning rate scheduling
2. Try different augmentation strategies
3. Experiment with other GNN architectures like GAT (Graph Attention Networks)
4. Implement ensemble methods